In [66]:
### SET UP
import pandas as pd
import os
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler


### SET UP
data = pd.read_csv('aes-2015-csv.csv')

# Use this section to navigate to appropriate directory - it is assumed that CSV data is in your Downloads folder
#os.chdir("Downloads")  
#print(os.getcwd())  

### INITIAL THOUGHTS AND DATA SET OBSERVATIONS
# Response Variable = Totak value which appears to be in millions of dollars
# Data set is total income (plus breakdowns) for a variety of industries which look like they have a hierarchical nature to them (as per Industry_aggregation_NZSIOC)
# Dataset is in an unusual format where each variable value is its own record in the DS - I will convert to tabular format for the sake of familiarity (I'm actually not sure if you can work with it straight like this)

# Data set needs to be restructured to tabular format & values converted to numeric
data['Value'] = pd.to_numeric(data['Value'], errors='coerce')
data_tabular = data.pivot_table(index='Industry_name_NZSIOC', columns='Variable_name', values='Value', aggfunc='first')
data_tabular.reset_index(inplace=True)

# Make sure everything looks like it has converted properly
#print(data_tabular.columns)
#print(data_tabular['Total income'].mean())

# Am just going to drop NaN values as I don't know anything about this data to choose the most appropriate imputation method
data_cleaned = data_tabular.dropna()
# Turns out that drops everything...
missing_values = data_tabular.isnull().sum()
#print(missing_values) 

sorted_columns = missing_values.sort_values()
top_cols = sorted_columns.head(30).index
data_top = data_tabular[top_cols]

# Now we can drop NaN values without removing the whole dataset as we've already excluded sparse vars.
data_cleaned_2 = data_top.dropna()
numeric_data = data_cleaned_2.select_dtypes(include=[np.number])

# Split into Response & Predictors
X = numeric_data.drop(columns=['Total income'])
y = numeric_data['Total income'] 

# Create a correlation matrix & drop predictors above 90% correlation
threshold = 0.9
correlation_matrix = X.corr()
upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > threshold)]
X = X.drop(columns=to_drop)

# Find 4 most correlated predictors to response var
corr_with_y = X.corrwith(y)
sorted_corr = corr_with_y.abs().sort_values(ascending=False)
top_4_correlated = sorted_corr.head(4)
top_4_variables = top_4_correlated.index
X = X[top_4_variables]

#################################################
###   Thoughts on selected 4 response vars   ####
#################################################
# I've selected my 4 response variables based on the fact that they are the top 4 highest correlated to the response var 
# The threshold for multicollinearity is arbitrary right now & I have not been able to have a good look around the data given I've been told this should take me 3 hours.
# The 4 predictors Closing Stocks, Current Ratio, Sales of Goods... and Quick Ratio have been the chosen 4 as they seem like a good place to start exploring

# Next steps is to start modelling so we'd want to split into test and train datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

LRmod = LinearRegression()
scaler = StandardScaler()

model.fit(X_train, y_train)

# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)
# model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test)
# y_pred = model.predict(X_test_scaled)

r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"R2: {r2:.2f}")
print(f"MSE: {mse:.2f}")

### Scaled & Non scaled models produce virtually the same results.
### The model is not good... I'm going to assume as I've been asked to keep it to 3 hours I shouldn't be going into refinements
### ...or the test is supposed to be on data that is painful to work with and returns something disappointing haha
### My gut feeling why this data isn't modelling well is because there is a lot of variation between industries. There probably wasn't enough data to look at each industry separately, but my thoughts are heading to an ensembe of separate models for each industry.


R2: -4.27
MSE: 643432406.75
